In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from sklearn.model_selection import train_test_split
import json

2024-12-29 17:54:50.923229: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-29 17:54:50.923665: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-29 17:54:50.925840: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-29 17:54:50.932383: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735475090.943472  487835 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735475090.94

In [2]:
ratings = pd.read_csv('../backend/datasets/ratings.csv')
keywords_data = pd.read_csv('../backend/datasets/keywords.csv', delimiter=',')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [4]:
keywords_data.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [5]:
ratings.describe()

,userId,movieId,rating,timestamp
count,2.602429e+07,2.602429e+07,2.602429e+07,2.602429e+07
mean,1.350371e+05,1.584911e+04,3.528090e+00,1.171258e+09
std,7.817620e+04,3.108526e+04,1.065443e+00,2.052889e+08
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08
25%,6.716400e+04,1.073000e+03,3.000000e+00,9.907545e+08
50%,1.351630e+05,2.583000e+03,3.500000e+00,1.151716e+09
75%,2.026930e+05,6.503000e+03,4.000000e+00,1.357578e+09
max,2.708960e+05,1.762750e+05,5.000000e+00,1.501830e+09


In [6]:
keywords_data.describe()

,id
count,46419.000000
mean,109769.951873
std,113045.780256
min,2.000000
25%,26810.500000
50%,61198.000000
75%,159908.500000
max,469172.000000


In [7]:
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [8]:
keywords_data.isnull().sum()

id          0
keywords    0
dtype: int64

## DATA PREPROCESSING

In [9]:
ratings = ratings.drop_duplicates()

In [10]:
ratings

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990


In [11]:
print(keywords_data.shape)
keywords_data = keywords_data.drop_duplicates(subset=['id'])
print(keywords_data.shape)

(46419, 2)
(45432, 2)


In [12]:
keywords_data

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [13]:
keywords_data.reset_index(drop=True, inplace=True)

In [14]:
keywords_data

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
45427,439050,"[{'id': 10703, 'name': 'tragic love'}]"
45428,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
45429,67758,[]
45430,227506,[]


In [15]:
keywords_data.describe()

,id
count,45432.000000
mean,108368.766750
std,112472.571213
min,2.000000
25%,26459.250000
50%,59995.500000
75%,157350.250000
max,469172.000000


In [16]:
ratings.describe()

,userId,movieId,rating,timestamp
count,2.602429e+07,2.602429e+07,2.602429e+07,2.602429e+07
mean,1.350371e+05,1.584911e+04,3.528090e+00,1.171258e+09
std,7.817620e+04,3.108526e+04,1.065443e+00,2.052889e+08
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08
25%,6.716400e+04,1.073000e+03,3.000000e+00,9.907545e+08
50%,1.351630e+05,2.583000e+03,3.500000e+00,1.151716e+09
75%,2.026930e+05,6.503000e+03,4.000000e+00,1.357578e+09
max,2.708960e+05,1.762750e+05,5.000000e+00,1.501830e+09


In [17]:
ratings['movieId'].max()

np.int64(176275)

In [18]:
keywords_data['id'].nunique()

45432

In [19]:
ratings['movieId'].nunique()

45115

In [20]:
#NOTE: total no of keywords column > total no of movies

### EXPLORING KEYWORDS DATA
- parsing keywords directly to json causing error because keys(id and name) are not properly nested in side double quote

In [21]:
keywords_data.shape

(45432, 2)

In [22]:
keywords_data.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [23]:
keywords_data['keywords'].iloc[0]

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [24]:
# json.loads(keywords_data['keywords'].iloc[1].replace("\'id\'",'"id"').replace("\'name\'", "name"))

In [25]:
data = keywords_data['keywords'].iloc[17]

In [26]:
print(data.replace('\"s',"\'s"))

[{'id': 612, 'name': 'hotel'}, {'id': 613, 'name': "new year's eve"}, {'id': 616, 'name': 'witch'}, {'id': 622, 'name': 'bet'}, {'id': 922, 'name': 'hotel room'}, {'id': 2700, 'name': 'sperm'}, {'id': 12670, 'name': 'los angeles'}, {'id': 160488, 'name': 'hoodlum'}, {'id': 187056, 'name': 'woman director'}, {'id': 198129, 'name': 'episode film'}]


In [27]:
# find currupted data before processing and after processing

# data = keywords_data['keywords'].iloc[1].replace("\'", '\"')

def convert_to_json(data):
    return data.replace("\'", '\"')

def is_currupt(data):
    curpt = False
    try:
        json.loads(data)
    except:
        curpt = True
    return curpt

In [28]:
keywords_data['keywords'].iloc[1]

'[{\'id\': 10090, \'name\': \'board game\'}, {\'id\': 10941, \'name\': \'disappearance\'}, {\'id\': 15101, \'name\': "based on children\'s book"}, {\'id\': 33467, \'name\': \'new home\'}, {\'id\': 158086, \'name\': \'recluse\'}, {\'id\': 158091, \'name\': \'giant insect\'}]'

In [177]:
# json.loads(keywords_data['keywords'].iloc[1].replace("\'s ", " ")).replace("\'", '\"')
# giving error

In [29]:
keywords_data['keywords'] = keywords_data['keywords'].apply(convert_to_json)

In [30]:
keywords_data['keywords'].iloc[1]

'[{"id": 10090, "name": "board game"}, {"id": 10941, "name": "disappearance"}, {"id": 15101, "name": "based on children"s book"}, {"id": 33467, "name": "new home"}, {"id": 158086, "name": "recluse"}, {"id": 158091, "name": "giant insect"}]'

In [31]:
keywords_data['keywords'].iloc[3]

'[{"id": 818, "name": "based on novel"}, {"id": 10131, "name": "interracial relationship"}, {"id": 14768, "name": "single mother"}, {"id": 15160, "name": "divorce"}, {"id": 33455, "name": "chick flick"}]'

In [32]:
keywords_data['currupted'] = keywords_data['keywords'].apply(is_currupt)

In [33]:
keywords_data['currupted']

0        False
1         True
2        False
3        False
4        False
         ...  
45427    False
45428    False
45429    False
45430    False
45431    False
Name: currupted, Length: 45432, dtype: bool

### PROCESSING KEYWORDS DATA

In [34]:
def find_currupted_index(keywords_data):
    list_of_currupted_rows = []
    index = 0
    for data in keywords_data['currupted']:
        if data == True:
            list_of_currupted_rows.append(index)
        index += 1
    return list_of_currupted_rows

In [35]:
keywords_data.shape

(45432, 3)

In [36]:
new_keywords_df = keywords_data

In [37]:
new_keywords_df

,id,keywords,currupted
0,862,"[{""id"": 931, ""name"": ""jealousy""}, {""id"": 4290,...",False
1,8844,"[{""id"": 10090, ""name"": ""board game""}, {""id"": 1...",True
2,15602,"[{""id"": 1495, ""name"": ""fishing""}, {""id"": 12392...",False
3,31357,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",False
4,11862,"[{""id"": 1009, ""name"": ""baby""}, {""id"": 1599, ""n...",False
...,...,...,...
45427,439050,"[{""id"": 10703, ""name"": ""tragic love""}]",False
45428,111109,"[{""id"": 2679, ""name"": ""artist""}, {""id"": 14531,...",False
45429,67758,[],False
45430,227506,[],False


In [38]:
new_keywords_df.reset_index(drop=True, inplace=True)

In [39]:
new_keywords_df

,id,keywords,currupted
0,862,"[{""id"": 931, ""name"": ""jealousy""}, {""id"": 4290,...",False
1,8844,"[{""id"": 10090, ""name"": ""board game""}, {""id"": 1...",True
2,15602,"[{""id"": 1495, ""name"": ""fishing""}, {""id"": 12392...",False
3,31357,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",False
4,11862,"[{""id"": 1009, ""name"": ""baby""}, {""id"": 1599, ""n...",False
...,...,...,...
45427,439050,"[{""id"": 10703, ""name"": ""tragic love""}]",False
45428,111109,"[{""id"": 2679, ""name"": ""artist""}, {""id"": 14531,...",False
45429,67758,[],False
45430,227506,[],False


In [40]:
new_keywords_df.shape

(45432, 3)

In [41]:
new_keywords_df.head()

,id,keywords,currupted
0,862,"[{""id"": 931, ""name"": ""jealousy""}, {""id"": 4290,...",False
1,8844,"[{""id"": 10090, ""name"": ""board game""}, {""id"": 1...",True
2,15602,"[{""id"": 1495, ""name"": ""fishing""}, {""id"": 12392...",False
3,31357,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",False
4,11862,"[{""id"": 1009, ""name"": ""baby""}, {""id"": 1599, ""n...",False


In [42]:
list_of_currupted_rows = find_currupted_index(new_keywords_df)

In [172]:
# print(list_of_currupted_rows)

In [44]:
len(list_of_currupted_rows)

628

In [45]:
# new_keywords_df.drop(labels=list_of_currupted_rows, axis=0, inplace=True)

# above funciton is not working may be because of long list of indexes
new_keywords_df = new_keywords_df[new_keywords_df['currupted'] == False]

In [46]:
new_keywords_df.reset_index(drop=True, inplace=True)

In [47]:
keywords_data.iloc[44803]

id                                  411007
keywords     [{"id": 974, "name": "bank"}]
currupted                            False
Name: 44803, dtype: object

In [48]:
new_keywords_df.iloc[44802]

id           227506
keywords         []
currupted     False
Name: 44802, dtype: object

In [49]:
new_keywords_df

,id,keywords,currupted
0,862,"[{""id"": 931, ""name"": ""jealousy""}, {""id"": 4290,...",False
1,15602,"[{""id"": 1495, ""name"": ""fishing""}, {""id"": 12392...",False
2,31357,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",False
3,11862,"[{""id"": 1009, ""name"": ""baby""}, {""id"": 1599, ""n...",False
4,949,"[{""id"": 642, ""name"": ""robbery""}, {""id"": 703, ""...",False
...,...,...,...
44799,439050,"[{""id"": 10703, ""name"": ""tragic love""}]",False
44800,111109,"[{""id"": 2679, ""name"": ""artist""}, {""id"": 14531,...",False
44801,67758,[],False
44802,227506,[],False


In [50]:
# we have eliminated all currupted rows, now we have to extract list of keywords from keywords column

In [51]:
# first remove currupted column

In [52]:
new_keywords_df.drop(columns=['currupted'], inplace=True)

/tmp/ipykernel_487835/501662641.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_keywords_df.drop(columns=['currupted'], inplace=True)


In [53]:
new_keywords_df

,id,keywords
0,862,"[{""id"": 931, ""name"": ""jealousy""}, {""id"": 4290,..."
1,15602,"[{""id"": 1495, ""name"": ""fishing""}, {""id"": 12392..."
2,31357,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."
3,11862,"[{""id"": 1009, ""name"": ""baby""}, {""id"": 1599, ""n..."
4,949,"[{""id"": 642, ""name"": ""robbery""}, {""id"": 703, ""..."
...,...,...
44799,439050,"[{""id"": 10703, ""name"": ""tragic love""}]"
44800,111109,"[{""id"": 2679, ""name"": ""artist""}, {""id"": 14531,..."
44801,67758,[]
44802,227506,[]


In [54]:
new_keywords_df.shape

(44804, 2)

In [55]:
new_keywords_df['keywords'].iloc[44803] == '[]'

True

In [56]:
# empty_list = []
# for entry in new_keywords_df['keywords']:
#     if entry == '[]':
#         empty_list.append(entry.index)
new_keywords_df = new_keywords_df[new_keywords_df['keywords'].apply(lambda entry : entry != '[]')]

In [57]:
new_keywords_df.shape

(30464, 2)

In [58]:
new_keywords_df

,id,keywords
0,862,"[{""id"": 931, ""name"": ""jealousy""}, {""id"": 4290,..."
1,15602,"[{""id"": 1495, ""name"": ""fishing""}, {""id"": 12392..."
2,31357,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."
3,11862,"[{""id"": 1009, ""name"": ""baby""}, {""id"": 1599, ""n..."
4,949,"[{""id"": 642, ""name"": ""robbery""}, {""id"": 703, ""..."
...,...,...
44794,84419,"[{""id"": 9748, ""name"": ""revenge""}, {""id"": 9826,..."
44795,390959,"[{""id"": 224180, ""name"": ""blair witch""}]"
44796,289923,"[{""id"": 616, ""name"": ""witch""}, {""id"": 2035, ""n..."
44799,439050,"[{""id"": 10703, ""name"": ""tragic love""}]"


In [59]:
new_keywords_df.reset_index(drop=True, inplace=True)

In [60]:
new_keywords_df

,id,keywords
0,862,"[{""id"": 931, ""name"": ""jealousy""}, {""id"": 4290,..."
1,15602,"[{""id"": 1495, ""name"": ""fishing""}, {""id"": 12392..."
2,31357,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."
3,11862,"[{""id"": 1009, ""name"": ""baby""}, {""id"": 1599, ""n..."
4,949,"[{""id"": 642, ""name"": ""robbery""}, {""id"": 703, ""..."
...,...,...
30459,84419,"[{""id"": 9748, ""name"": ""revenge""}, {""id"": 9826,..."
30460,390959,"[{""id"": 224180, ""name"": ""blair witch""}]"
30461,289923,"[{""id"": 616, ""name"": ""witch""}, {""id"": 2035, ""n..."
30462,439050,"[{""id"": 10703, ""name"": ""tragic love""}]"


In [61]:
# Now we extract keywords name from keywords column

In [62]:
new_keywords_df['keywords'] = new_keywords_df['keywords'].apply(lambda x: [entry["name"] for entry in json.loads(x)])

# new_keywords_df = new_keywords_df['keywords'].apply(lambda x: [entry["name"] for entry in json.load(x)])

/tmp/ipykernel_487835/1692557842.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_keywords_df['keywords'] = new_keywords_df['keywords'].apply(lambda x: [entry["name"] for entry in json.loads(x)])


In [63]:
new_keywords_df.head()

,id,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,15602,"[fishing, best friend, duringcreditsstinger, o..."
2,31357,"[based on novel, interracial relationship, sin..."
3,11862,"[baby, midlife crisis, confidence, aging, daug..."
4,949,"[robbery, detective, bank, obsession, chase, s..."


In [64]:
import pickle
pickle.dump(new_keywords_df,open('../backend/processed_data/keywords_info.pkl', 'wb'))

#### 🤣 finding the problem took my whole day
#### problem some data contain \' and some contain '
#### 👉 keywords_data['keywords'].iloc[1]
#### '[{\'id\': 10090, \'name\': \'board game\'}, {\'id\': 10941, \'name\': \'disappearance\'}, {\'id\': 15101, \'name\': "based on children\'s book"}, {\'id\': 33467, \'name\': \'new home\'}, {\'id\': 158086, \'name\': \'recluse\'}, {\'id\': 158091, \'name\': \'giant insect\'}]'
#### keywords_data['keywords'].iloc[0]
#### "[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"


#### edit: again took whole day 😡
#### data got currupted
#### value inside "name" contains double quote also get converted on converting the single quote to double quote.
#### eg. children's -> children"s in second row -> \'name\': "based on children\'s book" 🤬
#### it is a severe problem 👹 without solving it, json will get delimitter error
#### try converting \'name\' to 'name' and \'id'\ to 'id'

#### result : we need to remove the rows that got currpted after converting \' to " or use regular expression to remove correct the data

#### later i found keywords have empty json data so i filtered them
#### my reaction: finally i can proceed 💆

In [65]:
# Next we will take only those row from ratings dataframe which contain movie keyword in new_keywords_df

In [66]:
# create map of entries present in new_keywords_df to get keywords list easily

In [112]:
my_map = list(enumerate(keywords_data['id']))
mymap = {value: key for key, value in my_map}

In [68]:
# we will use this list to feed neurons so that they will understand the pattern

### PROCESSING RATINGS DATA

In [69]:
# print(mymap)

In [138]:
new_ratings_df = pd.DataFrame(ratings)

In [139]:
def myfilter(movieId):
    if movieId in mymap:
        return True
    else:
        return False

In [140]:
new_ratings_df = new_ratings_df[new_ratings_df['movieId'].apply(myfilter)]

In [141]:
if 1221 in mymap:
    print("exist")

In [142]:
new_ratings_df.shape

(11436568, 4)

In [143]:
ratings.shape

(26024289, 4)

In [144]:
last_key, last_value = next(iter(mymap.items()))
print(f"Last key: {last_key}, Last value: {last_value}")

Last key: 862, Last value: 0


In [157]:
# checking keys are integer type
for key, val in mymap.items():
    if type(key) != int:
        print(f"Invalid type {type(key)} at index {val}")

In [159]:
# checking movieId in new_ratings_df are integer
for movie_id in new_ratings_df['movieId']:
    if type(movie_id) != int:
        print(f"movieId {movie_id} is {type(movie_id)}")

In [160]:
l = set()
def extract_keywords(movieId):
    try:
        return new_keywords_df['keywords'].iloc[mymap[movieId]]
    except IndexError as e:
        l.add(movieId)
        l.add(type(movieId))

In [146]:
new_ratings_df['keywords'] = new_ratings_df['movieId'].apply(extract_keywords)

In [147]:
len(l)

1589

In [173]:
# print(l, end=' ')

In [175]:
# if 131074 in mymap:
#     print("exist")

In [164]:
# new_keywords_df['keywords'].iloc[mymap[131074]]

# -> result : IndexError

# Important ->
# Conclusion : list of data inside l are not available in new_keywords_df which result in single positional indexer in out of bound eror

In [165]:
new_ratings_df.shape

(11436568, 5)

In [166]:
new_ratings_df.head()

,userId,movieId,rating,timestamp,keywords
0,1,110,1.0,1425941529,"[newspaper, adversary, reporter, experience]"
1,1,147,4.5,1425942435,"[odessa, revenge]"
2,1,858,5.0,1425941523,"[letter, arkansas, childbirth, biracial, matri..."
4,1,1246,5.0,1425941556,"[friendship, romantic comedy, teenager, lovesi..."
5,1,1968,4.0,1425942148,"[pop, pop culture, pop star]"


In [168]:
# store processed ratings data
pickle.dump(new_ratings_df, open('../backend/processed_data/new_ratings_df.pkl','wb'))

In [171]:
!ls ../backend/processed_data/

keywords_info.pkl  new_ratings_df.pkl	  trending_movies.pkl
movies.pkl	   similarity_matrix.pkl


#### RESULT : We have processed ratings and keywords data in new_ratings_df
- TODO: extract movies keyword from movies dataset using TF_IDF algo in overview column

## Modeling

We have processed the ratings and keywords and store it in new_ratings_df. we have to now create a model to understand the pattern and using this we will build our recommendation system.

# Recommendation System

A **Recommendation System** is a machine learning model designed to suggest items to users based on their preferences, interests, and item similarity. It helps reduce the time users spend manually for searching the items of his interest. Recommendation systems are widely used in various domains like e-commerce, streaming platforms, and social media.
Recommendation System mainly categories into 3 parts.
1. **Popularity-Based Recommendation**
2. **Content-Based Recommendation**
3. **Collaborative Filtering**

## 1. **Popularity Based Recommendation:**
This type of recommendation system suggests items that are trending or popular among all users, irrespective of their individual preferences.
For example:

- The trending section on Netflix recommends shows or movies that are widely viewed by other users.
- The "Trending Now" section on X (formerly Twitter) highlights trending posts or topics.
    
## 2. **Content based filtering**:
This Model used the feature of the currently visited or liked item by the user like if a user like a movie of genre action and comedy. Model will recommmend him similar movie containing same tags (action and comedy). To do show we use vectorization technique(eg. TF-IDF, count_vectorizer) and cosine similarity.
## 3. **Collaborative filtering**:
It used the idea of similarity in preference and co-occurence of the item.
It is called collaborative because it used collabortion of various user and items to suggest items.
Collaborative Filtering is further divided into:

1. **Neighbourhood based approach**
2. **Model Based Approaches**

### 3.1 **Neighbourhood based approach:**
This method identifies users or items with similar preferences.
- **User-Based Filtering:**

    - Finds users (neighbors) with similar preferences or behavior.
    - Predicts how likely a user is to like an item based on the preferences of similar users.
    - Example: If User A and User B have rated the same movies highly, a movie liked by User A but not yet watched by User B may be recommended to User B.

- **Item-Based Filtering:**

    - Focuses on the similarity between items based on user interactions.
    - Uses a co-occurrence matrix that records how often items are interacted with together.
    - Items are ranked and sorted based on similarity scores.
    - Example: If a user liked Movie A and Movie B, a new user liking Movie A might also be recommended Movie B.
User and item based filtering both uses nearest neighbour learning algorithm(**KNN**). As we know KNN is a run time algorithm and in any instant there is a million or billion of items and users, it will become memory consuming and takes lots of time. so during the run time, this is not a good approach in recommendation system.
User and Item based filtering both are classification method.

### 3.2 **Model Based Approach**
These approaches use advanced algorithms to build more scalable and accurate recommendation systems.
i) **Clustering-Based Filtering**
ii) **Matrix Factorization**
iii) **Deep Learning-Based Approaches**

In [179]:
class Neurons():
    def __init__(self, input_val, weight):
        self.input_val = input_val
        self. weight = weight
    def weighted_sum():
        weighted_sum = 0
        for val, wg in self.input_val, self.weight:
            weighted_sum += val * wg
        return weighted_sum
    def apply_relu():
        relu = 1/(1 + power(ex, -x))